<a href="https://colab.research.google.com/github/pascualcam/aiml_code/blob/main/ch4/ch4_parallelized_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
import tensorflow_datasets as tfds
import multiprocessing

MODEL ARCHITECTURE

In [18]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

def create_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu',
                                      input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])


  model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

EXTRACT

In [19]:
train_data = tfds.load('cats_vs_dogs', split = 'train', with_info = 'true')

In [20]:
file_pattern = f'/root/tensorflow_datasets/cats_vs_dogs/4.0.0/cats_vs_dogs-train.tfrecord*'
files = tf.data.Dataset.list_files(file_pattern)

In [21]:
# set max degree of parallelism
train_dataset = files.interleave(
    tf.data.TFRecordDataset,
    cycle_length = 4,  # input elements processed concurrently
    num_parallel_calls = tf.data.experimental.AUTOTUNE  # parallel calls to execute
)

TRANSFORM

In [22]:
def read_tfrecord(serialized_example):
  #
  feature_description = {
      "image": tf.io.FixedLenFeature((), tf.string, ""),
      "label": tf.io.FixedLenFeature((), tf.int64, -1)
  }
  #
  example = tf.io.parse_single_example(
      serialized_example, feature_description
  )
  image = tf.io.decode_jpeg(example['image'], channels = 3)
  image = tf.cast(image, tf.float32)
  image = image / 255
  image = tf.image.resize(image, (300, 300))
  return image, example['label']

In [23]:
cores = multiprocessing.cpu_count()
print(cores)
train_dataset = train_dataset.map(read_tfrecord, num_parallel_calls = cores)
train_dataset = train_dataset.cache()

2


LOAD

In [24]:
# shuffle and batch the data
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
# train the model
model = create_model()
model.fit(
    train_dataset,
    epochs = 10,
    verbose = 1
)

Epoch 1/10
    246/Unknown - 772s 3s/step - loss: 0.6821 - accuracy: 0.5556